---
title: Data Analysis
subtitle: Comprehensive Data Cleaning & Exploratory Analysis of Job Market Trends
author:
  - name: Group 11
    affiliations:
      - name: Boston University
        city: Boston
        state: MA
        
format: 
  html:
    bibliography: references.bib
    csl: csl/econometrica.csl
    toc: true
    css: styles.css
    html-math-method: katex
    embed-resources: false
    code-fold: true
jupyter: python3
execute:
   eval: true

---






# Introduction
This document outlines the data cleaning process, including:
- Handling missing values
- Dropping unnecessary columns
- Deduplicating records

## Extracting the Dataset
The Lightcast Dataset has been extracted from the google drive using the gdown.


In [ ]:
import pandas as pd
import gdown
import matplotlib.pyplot as plt
import plotly.express as px
import gdown
import zipfile
import os


file_id = "1VNBTxArDMN2o9fJBDImaON6YUAyJGOU6" 
zip_file = "lightcast_job_postings.zip"  # Name of the downloaded ZIP file
csv_file = "./data/lightcast_job_postings.csv"  # Path to the CSV file

gdown.download(f"https://drive.google.com/uc?id={file_id}", zip_file, quiet=True)

with zipfile.ZipFile(zip_file, "r") as zip_ref:
    zip_ref.extractall("./data") 
df = pd.read_csv(csv_file)

## Dropping the Unnecessary columns Columns
There are 102 columns in the dataset and there are various columns which are unnecessary, so we are dropping those to clean the data and do better analysis


In [ ]:
columns_to_drop = [
    "ID", "URL", "ACTIVE_URLS", "DUPLICATES", "LAST_UPDATED_TIMESTAMP",
    "NAICS2", "NAICS3", "NAICS4", "NAICS5", "NAICS6",
    "SOC_2", "SOC_3", "SOC_5"
]

df.drop(columns=columns_to_drop, inplace=True)
print("Dropped unnecessary columns.")

## Removed the Duplicates and Handled the Missing Values


In [ ]:
import missingno as msno
import matplotlib.pyplot as plt


df.columns = df.columns.str.upper().str.strip()  # Normalize column names

msno.heatmap(df)
plt.title("Missing Values Heatmap")
plt.show()

# Drop columns with >50% missing values
df.dropna(thresh=len(df) * 0.5, axis=1, inplace=True)

df = df.drop_duplicates(subset=["TITLE", "COMPANY", "LOCATION", "POSTED"])
print("Duplicates removed.")
print("✅ Missing value handling complete.")

## Exploratory Data Analysis

### Data Analytics & Business Analytics Job Trend


In [ ]:
# identifying data analyst jobs by keyword searching
keywords = ['Data Analyst', 'Business Analyst', 'Data Engineering', 'Deep Learning',
            'Data Science', 'Data Analysis','Data Analytics',  'Market Research Analyst' 
            'LLM', 'Language Model', 'NLP', 'Natural Language Processing',
            'Computer Vision', 'Business Intelligence Analyst', 'Quantitative Analyst', 'Operations Analyst']

match = lambda col: df[col].str.contains('|'.join(keywords), case=False, na=False)

df['DATA_ANALYST_JOB'] = match('TITLE_NAME') \
             | match('SKILLS_NAME') \
             | match('SPECIALIZED_SKILLS_NAME') 
df['DATA_ANALYST_JOB'].value_counts()

In [ ]:
import plotly.express as px

df_grouped = df.groupby(['DATA_ANALYST_JOB', 'NAICS2_NAME']).size().reset_index(name='Job_Count')
# Define custom colors for 'True' and 'False' categories
color_map = {
    False: "#d62728" ,   # Red
    True: "#3BB143 "   # Green
}


fig = px.bar(df_grouped, 
             x='NAICS2_NAME', 
             y='Job_Count',
             color='DATA_ANALYST_JOB',
             title="Data Analytics & Business Analytics Job Trends",
              labels={'NAICS2_NAME': 'Industry', 'Job_Count': 'Number of Jobs'},
             barmode='group',
             color_discrete_map=color_map  # Custom colors
            )

fig.update_layout(
    yaxis=dict(
        title="Number of Jobs", 
        range=[0, df_grouped['Job_Count'].max() * 1.2]  # Increase y-axis height
    ),
    height=700,  # Make the figure taller
    xaxis=dict(
        tickangle=-45  # Rotate x-axis labels for better readability
    )
)

## Data Analytics & Business Analytics Job Trends
<iframe src="figures/plot1_jobtrends.html" width="100%" height="600px" style="border:none; display:block; margin:auto; min-height: 600px;"></iframe>

### Top 10 Industries Hiring the Data Analysts


In [ ]:
import plotly.express as px

# Identify the top 2 industries with the most data analyst jobs
top_industries = df_grouped.groupby("NAICS2_NAME")["Job_Count"].sum().nlargest(10).index

# Filter the original grouped DataFrame, not the raw df
df_top_industries = df_grouped[df_grouped["NAICS2_NAME"].isin(top_industries)]

# Create the bar chart to analyze job counts within these industries
fig = px.bar(df_top_industries, 
             x="DATA_ANALYST_JOB", 
             y="Job_Count", 
             color="NAICS2_NAME",
             title="Top 10 Industries Hiring Data Analysts",
             labels={'DATA_ANALYST_JOB': 'Job Title', 'Job_Count': 'Number of Jobs'},
             barmode='group')

## Data Analytics & Business Analytics Job Trends
<iframe src="figures/plot2industries.html" width="100%" height="600px" style="border:none; display:block; margin:auto;"></iframe>


In [ ]:
# No output to avoid blank spaces

### Remote Vs On-site Data Analyst Jobs in Finance & Healthcare Sector


In [ ]:
import plotly.express as px

# Define target industries
target_industries = ["Finance and Insurance", "Health Care and Social Assistance"]

# Filter the original DataFrame (df) instead of df_grouped
df_remote = df[df["NAICS2_NAME"].isin(target_industries)]

# Count occurrences of each remote type in these industries
df_remote_grouped = df_remote.groupby("REMOTE_TYPE_NAME").size().reset_index(name="Count")

# Create a pie chart
fig = px.pie(df_remote_grouped, 
             names="REMOTE_TYPE_NAME", 
             values="Count", 
             title="Remote vs. On-Site Data Analyst Jobs in Finance & Healthcare",
             color="REMOTE_TYPE_NAME", 
             color_discrete_map={"Remote": "#1f77b4", "On-Site": "#d62728", "Hybrid": "#2ca02c"})

<iframe src="figures/plot3pie.html" width="100%" height="600px" frameborder="0"></iframe>